In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
import re

In [85]:
# Load dataset
ahs_data = pd.read_csv('ahs2023n.csv')
ahs_data.head(100)

,CONTROL,TOTROOMS,PERPOVLVL,JACPRIMARY,JACSECNDRY,JADEQUACY,JAIRRATE,JBATHEXCLU,JBATHROOMS,JBEDROOMS,...,JOBFUNDS23,JOBFUNDS24,JOBTYPE21,JOBTYPE22,JOBTYPE23,JOBTYPE24,JOBWORKYR21,JOBWORKYR22,JOBWORKYR23,JOBWORKYR24
0,'11000002',6,-6,'0','0','2','0','0','0','0',...,' ',' ',' ',' ',' ',' ',' ',' ',' ',' '
1,'11000003',4,199,'0','0','2','0','0','0','0',...,' ',' ',' ',' ',' ',' ',' ',' ',' ',' '
2,'11000005',7,501,'0','0','2','0','0','0','0',...,' ',' ',' ',' ',' ',' ',' ',' ',' ',' '
3,'11000006',5,232,'0','0','2','0','0','0','0',...,' ',' ',' ',' ',' ',' ',' ',' ',' ',' '
4,'11000008',3,231,'2','2','2','0','0','0','0',...,' ',' ',' ',' ',' ',' ',' ',' ',' ',' '
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,'11000213',2,1,'0','0','2','0','0','0','0',...,' ',' ',' ',' ',' ',' ',' ',' ',' ',' '
96,'11000214',5,74,'0','0','2','0','0','0','0',...,' ',' ',' ',' ',' ',' ',' ',' ',' ',' '
97,'11000215',3,54,'0','0','2','0','0','0','0',...,' ',' ',' ',' ',' ',' ',' ',' ',' ',' '
98,'11000216',5,501,'0','0','2','0','0','0','0',...,' ',' ',' ',' ',' ',' ',' ',' ',' ',' '


In [86]:
for col in ahs_data.columns:
    if ahs_data[col].dtype == 'object':
        ahs_data[col] = ahs_data[col].str.replace("'", "", regex=False)
        ahs_data[col] = ahs_data[col].replace(['', ' ', '  '], np.nan)
        ahs_data[col] = ahs_data[col].str.strip()
        ahs_data[col] = ahs_data[col].replace('', np.nan)

numeric_cols = [col for col in ahs_data.columns if any(x in col for x in 
                ['JOB', 'FUNDS', 'TYPE', 'WORKYR', 'BEDROOMS', 'BATHROOMS'])]

for col in numeric_cols:
    ahs_data[col] = pd.to_numeric(ahs_data[col], errors='coerce')
print("\nMissing Values per Column:")
print(ahs_data.isnull().sum().sort_values(ascending=False).head(20))


Missing Values per Column:
JOBWORKYR24    55668
JSSP15         55668
JWAGP15        55668
MAR14          55668
MAR15          55668
MIL14          55668
MIL15          55668
MLPA14         55668
MLPA15         55668
MLPB14         55668
MLPB15         55668
MLPCD14        55668
MLPCD15        55668
MLPE14         55668
MLPE15         55668
MLPFG14        55668
MLPFG15        55668
MLPH14         55668
MLPH15         55668
MLPI14         55668
dtype: int64


In [87]:
# Keep columns with at least 50% non-null
threshold = len(ahs_data) % .5
ahs_data = ahs_data.loc[:, ahs_data.isnull().sum() <= threshold]
print(ahs_data.shape)

(55669, 1429)


In [80]:
# Finds the Market Value Column
candidates = [c for c in ahs_data.columns if re.search(r'price|value|val|sale|market|home|worth', c, re.I)]
print(candidates)
ahs_data[candidates].head()

['JFIRSTHOME', 'JHHPRNTHOME', 'JHMRSALE', 'JMARKETVAL', 'JRMHOME', 'RMHOME', 'HMRSALE', 'HHPRNTHOME', 'FIRSTHOME']


,JFIRSTHOME,JHHPRNTHOME,JHMRSALE,JMARKETVAL,JRMHOME,RMHOME,HMRSALE,HHPRNTHOME,FIRSTHOME
0,0,0,0,0,0,-6,-6,-6,-6
1,0,0,0,0,0,1,-6,1,-6
2,0,0,0,0,0,-6,2,2,2
3,0,0,0,0,0,-6,2,1,2
4,0,0,0,0,0,-6,-6,2,-6


In [77]:
ahs_data

,CONTROL,TOTROOMS,PERPOVLVL,JACPRIMARY,JACSECNDRY,JADEQUACY,JAIRRATE,JBATHEXCLU,JBATHROOMS,JBEDROOMS,...,PROTAXAMT,INSURAMT,HOAAMT,LOTAMT,MAINTAMT,MORTAMT,HINCP,FINCP,REMODAMT,TOTHCAMT
0,11000002,6,-6,0,0,2,0,0,0,0,...,-6,-6,-6,-6,-6,-6,-6,-6,-6,-6
1,11000003,4,199,0,0,2,0,0,0,0,...,-6,13,-6,-6,-6,-6,48000,48000,-6,1093
2,11000005,7,501,0,0,2,0,0,0,0,...,245,267,21,-6,201,-6,292500,292500,500,810
3,11000006,5,232,0,0,2,0,0,0,0,...,122,59,0,-6,458,-6,56000,56000,20000,489
4,11000008,3,231,2,2,2,0,0,0,0,...,-6,41,-6,-6,-6,-6,36000,36000,-6,845
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55664,11100196,3,187,0,0,2,0,0,0,0,...,-6,7,-6,-6,-6,-6,27000,27000,-6,1817
55665,11100200,3,-6,0,0,2,0,0,0,0,...,-6,-6,-6,-6,-6,-6,-6,-6,-6,-6
55666,11100202,4,-6,0,0,3,0,0,0,0,...,-6,-6,-6,-6,-6,-6,-6,-6,-6,-6
55667,11100203,4,-6,0,0,2,0,0,0,0,...,-6,-6,-6,-6,-6,-6,-6,-6,-6,-6


In [79]:
# Drop the Market Value Column
ahs_data = ahs_data.drop(columns=['MARKETVAL'])
market_data = ahs_data["MARKETVAL"]
market_data

KeyError: "['MARKETVAL'] not found in axis"